In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

'c:\\Users\\jaesc2\\GitHub\\skforecast'

In [16]:
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregMultiSeriesCustom import ForecasterAutoregMultiSeriesCustom
from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
import numpy as np
from sklearn.linear_model import LinearRegression
import joblib
import inspect
import warnings

In [17]:
# Custom functions
def custom_pred(y):
    return np.mean(y)

def custom_weights(y):
    return np.ones(len(y))

def custom_weights2(y):
    return np.arange(1, len(y)+1)

In [4]:
# forecaster = ForecasterAutoregCustom(
#     regressor = LinearRegression(),
#     window_size=50,
#     fun_predictors= custom_pred,
#     weight_func= custom_weights
# )


forecaster = ForecasterAutoregMultiSeriesCustom(
    regressor = LinearRegression(),
    window_size=50,
    fun_predictors= custom_pred,
    weight_func= {'series 1': custom_weights, 'series 2': custom_weights2}
)

In [11]:
def save_forecaster(
    forecaster, 
    file_name: str,
    save_custom_functions: bool=True, 
    verbose: bool=True
) -> None:
    """
    Save forecaster model using joblib. If custom functions are used to create
    predictors or weights, they are saved as .py files.

    Parameters
    ----------
    forecaster: forecaster
        Forecaster created with skforecast library.
    file_name: str
        File name given to the object.
    save_custom_functions: bool, default `True`
        If True, save custom functions used in the forecaster (fun_predictors and
        weight_func) as .py files. Custom functions need to be available in the
        environment where the forecaster is going to be loaded.
    verbose: bool, default `True`
        Print summary about the forecaster saved.

    Returns
    -------
    None

    """

    # Save forecaster
    joblib.dump(forecaster, filename=file_name)

    if save_custom_functions:
        # Save custom functions to create predictors
        if hasattr(forecaster, 'fun_predictors') and forecaster.fun_predictors is not None:
            file_name = forecaster.fun_predictors.__name__ + '.py'
            with open(file_name, 'w') as file:
                file.write(inspect.getsource(forecaster.fun_predictors))

        # Save custom functions to create weights
        if hasattr(forecaster, 'weight_func') and forecaster.weight_func is not None:
            if isinstance(forecaster.weight_func, dict):
                for fun in set(forecaster.weight_func.values()):
                    file_name = fun.__name__ + '.py'
                    with open(file_name, 'w') as file:
                        file.write(inspect.getsource(fun))
            else:
                file_name = forecaster.weight_func.__name__ + '.py'
                with open(file_name, 'w') as file:
                    file.write(inspect.getsource(forecaster.weight_func))
    else:
        if ((hasattr(forecaster, 'fun_predictors') and forecaster.fun_predictors is not None)
          or (hasattr(forecaster, 'weight_func') and forecaster.weight_func is not None)):
            warnings.warn(
                ("Custom functions used to create predictors or weights are not saved. "
                 "To save them, set `save_custom_functions` to `True`.")
            )

    if verbose:
        forecaster.summary()

In [15]:
forecaster = ForecasterAutoregMultiSeriesCustom(
    regressor = LinearRegression(),
    window_size=50,
    fun_predictors= custom_pred,
    weight_func= {'series 1': custom_weights, 'series 2': custom_weights2}
)

TypeError: ForecasterAutoregMultiSeriesCustom.__init__() missing 1 required positional argument: 'fun_predictors'

In [18]:
forecaster = ForecasterAutoregMultiSeries(
    regressor = LinearRegression(),
    lags=50,
    # fun_predictors= custom_pred,
    weight_func= {'series 1': custom_weights, 'series 2': custom_weights2}
)

In [19]:
save_forecaster(forecaster, 'forecaster.joblib', save_custom_functions=False)

ForecasterAutoregMultiSeries 
Regressor: LinearRegression() 
Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50] 
Transformer for series: None 
Transformer for exog: None 
Window size: 50 
Series levels (names): None 
Series weights: None 
Weight function included: True 
Exogenous included: False 
Type of exogenous variable: None 
Exogenous variables names: None 
Training range: None 
Training index type: None 
Training index frequency: None 
Regressor parameters: {'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'positive': False} 
fit_kwargs: {} 
Creation date: 2023-12-08 22:53:58 
Last fit date: None 
Skforecast version: 0.12.0 
Python version: 3.11.5 
Forecaster id: None 



C:\Users\jaesc2\AppData\Local\Temp\ipykernel_12420\3387013849.py:54: UserWarning: Custom functions used to create predictors or weights are not saved. To save them, set `save_custom_functions` to `True`.
  warnings.warn(


In [20]:
import os

def create_predictors(y): # pragma: no cover
    """
    Create first 2 lags of a time series.
    """
    
    lags = y[-1:-3:-1]
    
    return lags

def custom_weights(y): # pragma: no cover
    """
    """
    return np.ones(len(y))

def custom_weights2(y): # pragma: no cover
    """
    """
    return np.arange(1, len(y)+1)

forecaster = ForecasterAutoregCustom(
                     regressor      = LinearRegression(),
                     fun_predictors = create_predictors,
                     window_size    = 3,
                     weight_func    = custom_weights
                 )
save_forecaster(forecaster=forecaster, file_name='forecaster.py', save_custom_functions=True)
os.remove('forecaster.py')

fun_predictors_file = forecaster.fun_predictors.__name__ + '.py'
assert os.path.exists(fun_predictors_file)
with open(fun_predictors_file, 'r') as file:
    assert inspect.getsource(forecaster.fun_predictors) == file.read()
os.remove(fun_predictors_file)

weight_func_file = forecaster.weight_func.__name__ + '.py'
assert os.path.exists(weight_func_file)
with open(weight_func_file, 'r') as file:
    assert inspect.getsource(forecaster.weight_func) == file.read()
os.remove(weight_func_file)

ForecasterAutoregCustom 
Regressor: LinearRegression() 
Predictors created with function: create_predictors 
Transformer for y: None 
Transformer for exog: None 
Window size: 3 
Weight function included: True 
Differentiation order: None 
Exogenous included: False 
Type of exogenous variable: None 
Exogenous variables names: None 
Training range: None 
Training index type: None 
Training index frequency: None 
Regressor parameters: {'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'positive': False} 
fit_kwargs: {} 
Creation date: 2023-12-08 23:01:34 
Last fit date: None 
Skforecast version: 0.12.0 
Python version: 3.11.5 
Forecaster id: None 

